In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from functools import reduce
import re

## 1. 데이터 로딩

In [19]:
# 각자의 데이터 디렉토리
data_dir = '../data/train/'

In [20]:
directors = pd.read_csv(data_dir+"directors.tsv", delimiter="\t")
genres = pd.read_csv(data_dir+"genres.tsv", delimiter="\t")
titles = pd.read_csv(data_dir+"titles.tsv", delimiter="\t")
writers = pd.read_csv(data_dir+"writers.tsv", delimiter="\t")
years = pd.read_csv(data_dir+"years.tsv", delimiter="\t")

train = pd.read_csv(data_dir+"train_ratings.csv")

#### 1-0 작가&감독 필모 수

In [21]:
# 감독의 작품 수
filmo_d = pd.DataFrame(directors.groupby('director').item.count()).reset_index(drop=False).rename(columns={'item':'count_director'})
directors = pd.merge(directors,filmo_d, how='left', left_on='director', right_on='director')

# 작가의 작품 수
filmo_w = pd.DataFrame(writers.groupby('writer').item.count()).reset_index(drop=False).rename(columns={'item':'count_writer'})
writers = pd.merge(writers,filmo_w, how='left', left_on='writer', right_on='writer')

### 1-1 감독, 장르, 제목, 작가, 년도로 나눠진 데이터 합치기

In [22]:
movie = reduce(
    lambda x, y: pd.merge(x, y, on="item", how="outer"),
    [directors, genres, titles, writers, years],
)
movie.head()

,item,director,count_director,genre,title,writer,count_writer,year
0,1237,nm0000005,19.0,Drama,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",nm0000005,18.0,1957.0
1,5147,nm0000005,19.0,Drama,Wild Strawberries (Smultronstället) (1957),nm0000005,18.0,1957.0
2,7327,nm0000005,19.0,Drama,Persona (1966),nm0000005,18.0,1966.0
3,2068,nm0000005,19.0,Drama,Fanny and Alexander (Fanny och Alexander) (1982),nm0000005,18.0,1982.0
4,2068,nm0000005,19.0,Fantasy,Fanny and Alexander (Fanny och Alexander) (1982),nm0000005,18.0,1982.0


### 1-2 합쳐진 데이터를 영화의 item id로 압축하기

In [23]:
grouped_movie = (
    movie.fillna(0)
    .groupby("item")
    .agg(set)
    .reset_index()
    .applymap(
        lambda x: (
            (list(x)[0] if len(list(x)) == 1 else list(x)) if isinstance(x, set) else x
        )
    )
)
grouped_movie = grouped_movie.replace([0], np.NaN)
grouped_movie["year"] = grouped_movie.year.fillna(0).astype("Int64").replace(0, np.NaN)
grouped_movie.head()

/tmp/ipykernel_542965/3916230186.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(


,item,director,count_director,genre,title,writer,count_writer,year
0,1,nm0005124,5.0,"[Comedy, Children, Animation, Fantasy, Adventure]",Toy Story (1995),"[nm0169505, nm0004056, nm0812513, nm0710020, n...","[5.0, 6.0, 8.0, 9.0, 10.0]",1995
1,2,nm0002653,9.0,"[Fantasy, Children, Adventure]",Jumanji (1995),"[nm0885575, nm0378144, nm0852430]","[2.0, 3.0, 7.0]",1995
2,3,nm0222043,7.0,"[Comedy, Romance]",Grumpier Old Men (1995),nm0425756,7.0,1995
3,4,nm0001845,3.0,"[Comedy, Drama, Romance]",Waiting to Exhale (1995),nm0060103,12.0,1995
4,5,nm0796124,4.0,Comedy,Father of the Bride Part II (1995),"[nm0583600, nm0329304, nm0796124, nm0352443]","[8.0, 9.0, 10.0]",1995


### 1-3 train 데이터 시청시간

- time: datetime 데이터 타입
- duration: timedelta 데이터 타입

In [24]:
# unix time -> datetime : 실행시간 9초 내외
train['time'] = pd.to_datetime(train['time'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
train['time'] = pd.to_datetime(train['time'])

In [25]:
# duration
train['duration'] = train.sort_values('time').groupby('user').time.diff(-1).abs() # by index (no worry)

## 2. 결측치 탐색

In [26]:
isna = grouped_movie.isna().sum()
isna

item                 0
director          1304
count_director    1304
genre                0
title                0
writer            1159
count_writer      1159
year                 8
dtype: int64

In [27]:
tmp = grouped_movie[grouped_movie["director"].isna()]
tmp = tmp.isna().sum()
tmp

item                 0
director          1304
count_director    1304
genre                0
title                0
writer             629
count_writer       629
year                 2
dtype: int64

### 2-1 년도 결측치 채우기

In [28]:
year_isna = grouped_movie[grouped_movie["year"].isna()]
year_isna.head()

,item,director,count_director,genre,title,writer,count_writer,year
1847,3310,nm0000122,9.0,"[Comedy, Drama]","Kid, The (1921)",nm0000122,10.0,<NA>
3455,6987,NaN,NaN,"[Horror, Fantasy, Crime]","Cabinet of Dr. Caligari, The (Cabinet des Dr. ...",nm0562346,3.0,<NA>
3456,6988,nm0000428,3.0,"[Drama, Romance]",Broken Blossoms or The Yellow Man and the Girl...,nm0000428,3.0,<NA>
3513,7065,nm0000428,3.0,"[War, Drama]","Birth of a Nation, The (1915)","[nm0940488, nm0000428]","[2.0, 3.0]",<NA>
3605,7243,nm0000428,3.0,Drama,Intolerance: Love's Struggle Throughout the Ag...,"[nm0002616, nm0115218, nm0940488, nm0000428]","[2.0, 3.0, 5.0]",<NA>


In [29]:
grouped_movie["year"].fillna(
    grouped_movie["title"].apply(
        lambda x: (
            re.search(r"(\d{4})", x).group()
            if re.search(r"(\d{4})", x) and "year" in grouped_movie.columns
            else np.NaN
        )
    ),
    inplace=True,
)
grouped_movie.head()

,item,director,count_director,genre,title,writer,count_writer,year
0,1,nm0005124,5.0,"[Comedy, Children, Animation, Fantasy, Adventure]",Toy Story (1995),"[nm0169505, nm0004056, nm0812513, nm0710020, n...","[5.0, 6.0, 8.0, 9.0, 10.0]",1995
1,2,nm0002653,9.0,"[Fantasy, Children, Adventure]",Jumanji (1995),"[nm0885575, nm0378144, nm0852430]","[2.0, 3.0, 7.0]",1995
2,3,nm0222043,7.0,"[Comedy, Romance]",Grumpier Old Men (1995),nm0425756,7.0,1995
3,4,nm0001845,3.0,"[Comedy, Drama, Romance]",Waiting to Exhale (1995),nm0060103,12.0,1995
4,5,nm0796124,4.0,Comedy,Father of the Bride Part II (1995),"[nm0583600, nm0329304, nm0796124, nm0352443]","[8.0, 9.0, 10.0]",1995


In [30]:
check_year = pd.DataFrame()
for i in year_isna["item"].tolist():
    check_year = pd.concat(
        [check_year.loc[:], grouped_movie[grouped_movie["item"] == i]]
    )
check_year.head()

,item,director,count_director,genre,title,writer,count_writer,year
1847,3310,nm0000122,9.0,"[Comedy, Drama]","Kid, The (1921)",nm0000122,10.0,1921
3455,6987,NaN,NaN,"[Horror, Fantasy, Crime]","Cabinet of Dr. Caligari, The (Cabinet des Dr. ...",nm0562346,3.0,1920
3456,6988,nm0000428,3.0,"[Drama, Romance]",Broken Blossoms or The Yellow Man and the Girl...,nm0000428,3.0,1919
3513,7065,nm0000428,3.0,"[War, Drama]","Birth of a Nation, The (1915)","[nm0940488, nm0000428]","[2.0, 3.0]",1915
3605,7243,nm0000428,3.0,Drama,Intolerance: Love's Struggle Throughout the Ag...,"[nm0002616, nm0115218, nm0940488, nm0000428]","[2.0, 3.0, 5.0]",1916


## 3. 데이터 저장

### 3-1 train_ratings.csv에 grouped_movie 정보 merge

In [31]:
trains = pd.merge(train, grouped_movie, on="item", how="inner")
trains = trains.sort_values(by="user")
trains.head()

,user,item,time,duration,director,count_director,genre,title,writer,count_writer,year
283464,11,880,2009-01-01 05:27:23,0 days 00:00:05,nm0001239,8.0,"[Sci-Fi, Thriller]","Island of Dr. Moreau, The (1996)","[nm0920229, nm0404396]","[2.0, 10.0]",1996
363517,11,2232,2009-01-01 05:43:10,0 days 00:00:04,nm0622112,4.0,"[Sci-Fi, Thriller, Mystery, Horror]",Cube (1997),nm0622112,3.0,1997
792847,11,2720,2009-01-01 23:58:00,0 days 00:27:10,NaN,NaN,"[Comedy, Children, Adventure, Action]",Inspector Gadget (1999),"[nm0672015, nm0647615]","[10.0, 4.0]",1999
522089,11,2642,2009-01-01 05:49:18,0 days 00:01:16,nm0504513,8.0,"[Sci-Fi, Adventure, Action]",Superman III (1983),"[nm0796950, nm0628174, nm0701374, nm0628058, n...","[10.0, 4.0, 6.0, 7.0]",1983
72138,11,8977,2009-01-01 04:30:16,0 days 00:00:38,nm0000231,16.0,"[War, Drama, Adventure, Action]",Alexander (2004),"[nm0477370, nm0436164, nm0000231]","[17.0, 2.0, 4.0]",2004


### 3-2 trains의 결측치

In [32]:
train_isna = trains.isna().sum()
train_isna

user                   0
item                   0
time                   0
duration           31360
director          301445
count_director    301445
genre                  0
title                  0
writer            312692
count_writer      312692
year                   0
dtype: int64

In [33]:
tmp = trains[trains["writer"].isna()]
tmp = tmp.isna().sum()
tmp

user                   0
item                   0
time                   0
duration            2403
director          118851
count_director    118851
genre                  0
title                  0
writer            312692
count_writer      312692
year                   0
dtype: int64

### 3-3 movie 정보와 user_movie interaction 정보 저장

In [34]:
grouped_movie.to_csv(data_dir + "movie.csv", index=False)
trains.to_csv(data_dir + "user_movie.csv", index=False)